# Finalise twitter topics
Finalises the twitter and topic data.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_params
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
TENANT_ID = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 11
OBJECT_TYPE = "tweets"

# OUTPUT
# This is legacy placement of the final parquet files and should be refactored at somepoint
FINAL_URL_TWEETS_TOPICS = None
RENAME_TOPIC_TO_CLASS = False

In [ ]:
run_params = run_params.general.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME)
    
url_config = {
    "YEAR_FILTER": int(YEAR_FILTER),
    "MONTH_FILTER": int(MONTH_FILTER),
    "OBJECT_TYPE": OBJECT_TYPE
}

TAGGING_RUNS_URL_TOPICS = run_params.art_url_reg.get_url("tagging_runs-topics", url_config)
TAGGING_RUNS_URL_TWEETS_FINAL = run_params.art_url_reg.get_url("tagging_runs-tweets_final", url_config)
TAGGING_RUNS_URL_TWEETS_TOPICS_FINAL = run_params.art_url_reg.get_url("tagging_runs-tweets_topics_final", url_config)
if not FINAL_URL_TWEETS_TOPICS:
    FINAL_URL_TWEETS_TOPICS = run_params.art_url_reg.get_url("final-tweets_topics", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_TOPICS,
TAGGING_RUNS_URL_TWEETS_FINAL,
TAGGING_RUNS_URL_TWEETS_TOPICS_FINAL,
FINAL_URL_TWEETS_TOPICS,
run_dt.dt,
sep='\n',
)

In [ ]:
tweets_df = artifacts.dataframes.get(TAGGING_RUNS_URL_TWEETS_FINAL).dataframe

In [ ]:
tweets_df.head()

In [ ]:
topics = artifacts.dataframes.get(TAGGING_RUNS_URL_TOPICS).dataframe

In [ ]:
topics.head()

In [ ]:
tweets_topics_final = finalise.join_topics_to_tweets(topics, tweets_df, rename_topic_to_class=RENAME_TOPIC_TO_CLASS)

In [ ]:
tweets_topics_final.head()

In [ ]:
tweets_topics_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FINAL_URL_TWEETS_TOPICS, tweets_topics_final)

In [ ]:
_ = artifacts.dataframes.persist(TAGGING_RUNS_URL_TWEETS_TOPICS_FINAL, tweets_topics_final)

In [ ]:
artifacts.dataframes.read_schema(TAGGING_RUNS_URL_TWEETS_TOPICS_FINAL)